In [189]:
import numpy as np 
import tensorflow as tf 
import os 
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras import layers, utils
from tqdm import tqdm
from skimage.measure import block_reduce
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy import ndimage


#from scipy.ndimage import gaussian_filter

In [190]:
def create_coordinate_data(pth):
    #np.random.seed(47) 
    
    data = [] # stores the coordinate data
    mapping = [] # maintain a mapping between file names and training/testing/validation designation 
    
    y_data = pd.read_csv(os.path.join(pth, 'parameters.csv'))

    y_regression = np.array(y_data[['J','Jp']])
    
    for k in tqdm(range(1,len(y_regression)+1)):
    #for k in range(1,100):
        train_test_valid = np.random.choice([0.0, 1.0, 2.0], p=[0.8, 0.1, 0.1]) # 0 = train, 1 = valid, 2 = test 
        
        j1 = (y_regression[k-1][0] - 20)/(75 - 20)
        j2 = (y_regression[k-1][1] + 30)/(10 + 30)

        c_q = np.array(pd.read_csv(os.path.join(pth, 'Klist_' + str(k) + '.csv'), header=None)).T
        c_E = np.array(pd.read_csv(os.path.join(pth, 'Elist_' + str(k) + '.csv'), header=None)).T
        c_sqw = np.array(pd.read_csv(os.path.join(pth, 'Sqw_withCorrectBroadening_' + str(k) + '.csv'), header=None)).T
        
        for j in range(c_sqw.shape[1]):
            for i in range(c_sqw.shape[0]):
                data.append([train_test_valid, c_q[i][0], c_q[i][1], c_q[i][2], float(c_E[j]/200), j1, j2, c_sqw[i,j]])
        
        mapping.append([k, train_test_valid])
        
    data = np.array(data)
    mapping = np.array(mapping)
    
    return data, mapping


In [193]:
base_path = '../data_simulation_2023/' 

data_slice_1, mapping_slice_1 = create_coordinate_data(os.path.join(base_path, 'AFM_PATH_1'))
data_slice_2, mapping_slice_2 = create_coordinate_data(os.path.join(base_path, 'AFM_PATH_2'))

100%|██████████| 600/600 [01:25<00:00,  7.02it/s]


In [194]:
data_both = np.vstack((data_slice_1, data_slice_2))
np.random.shuffle(data_both)

np.save('../converted_simulation_data_2023/data_both_paths_2023', data_both)
np.save('../converted_simulation_data_2023/mapping_path_1_2023', mapping_slice_1)
np.save('../converted_simulation_data_2023/mapping_path_2_2023', mapping_slice_2)

data_both = np.load('../converted_simulation_data_2023/data_both_paths_2023.npy')


In [195]:

train_idx = np.where(data_both[:,0] == 0.0)[0]
valid_idx = np.where(data_both[:,0] == 1.0)[0]
test_idx =  np.where(data_both[:,0] == 2.0)[0]

X_train = data_both[train_idx, 1:7]
X_valid =data_both[valid_idx, 1:7]
X_test = data_both[test_idx, 1:7]

y_train = data_both[train_idx, -1]
y_valid = data_both[valid_idx, -1]
y_test = data_both[test_idx, -1]
   

In [196]:
np.min(X_train[:,-1])

0.0013005687005949618

In [197]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(28148800, 6)
(3474800, 6)
(3416400, 6)
(28148800,)
(3474800,)
(3416400,)


In [198]:
np.savez('../converted_simulation_data_2023/neural_dataset.npz', train_x=X_train, valid_x=X_valid, test_x=X_test, train_y=y_train, valid_y=y_valid, test_y=y_test)